<div align="center">

# Лабораторна робота №2  

## Наука про дані: підготовчий етап  

</div>

### Мета роботи  
Ознайомитися з основними кроками по роботі з даними – workflow від постановки задачі до написання пояснювальної записки.  
Зрозуміти постановку задачі та природу даних, над якими виконуються аналітичні операції.  

---
<div align="center">
    
## Хід виконання роботи:
</div>

### -Імпорт бібліотек:


In [1]:
import os
import urllib.request
import pandas as pd
from datetime import datetime
import re
import math
print('Бібліотеки імпортовано без помилок')

Бібліотеки імпортовано без помилок


---
### -Завантаження та обробка файлів:

In [2]:
def download_vhi_data(region_id, directory):
    url = (f'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?'
           f'country=UKR&provinceID={region_id}&year1=1981&year2=2024&type=Mean')

    wp = urllib.request.urlopen(url)
    text = wp.read()

    now = datetime.now()
    timestamp = now.strftime("%d%m%Y%H%M%S")

    filename = f'NOAA_ID{region_id}_{timestamp}.csv'
    file_path = os.path.join(directory, filename)

    for f in os.listdir(directory):
        if f.startswith(f"NOAA_ID{region_id}_"):
            old_file_path = os.path.join(directory, f)
            os.remove(old_file_path)
            print(f"Видалено старий файл: {old_file_path}")

    with open(file_path, 'wb') as out:
        out.write(text)
    print(f"Завантажено новий файл: {file_path}")
    
    return file_path

def clean_html(text):
    return re.sub(r'<.*?>', '', str(text))

def process_file(file_path, region_id):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    df = pd.read_csv(file_path, header=1, names=headers, on_bad_lines='skip')

    if 'empty' in df.columns and df['empty'].isnull().all():
        df.drop(columns=['empty'], inplace=True)

    df['Year'] = df['Year'].astype(str).apply(clean_html)
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce').astype('Int64')

    df = df[df['VHI'] != -1]
    df['area'] = region_id
    
    return df

def download_and_process_all_regions(start_region, end_region, directory):
    all_dfs = []
    for region_id in range(start_region, end_region + 1):
        file_path = download_vhi_data(region_id, directory)
        df = process_file(file_path, region_id)
        if len(df) > 0:
            all_dfs.append(df)
    if all_dfs:
        full_df = pd.concat(all_dfs, ignore_index=True)
    else:
        full_df = pd.DataFrame()
    return full_df

def update_region_indices(df):
    index_mapping = {
        1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 12: 26, 13: 10, 14: 11, 15: 12, 16: 13, 17: 14, 18: 15, 19: 16, 
        20: 27, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5 
    }
    df['area'] = df['area'].replace(index_mapping)
    return df

if __name__ == "__main__":
    directory = './vhi_data'
    if not os.path.exists(directory):
        os.makedirs(directory)

    full_df = download_and_process_all_regions(1, 27, directory)
    full_df = update_region_indices(full_df)

    print("Перші 5 рядків об’єднаних даних:")
    print(full_df.head())


Завантажено новий файл: ./vhi_data\NOAA_ID1_14032025143522.csv
Завантажено новий файл: ./vhi_data\NOAA_ID2_14032025143523.csv
Завантажено новий файл: ./vhi_data\NOAA_ID3_14032025143524.csv
Завантажено новий файл: ./vhi_data\NOAA_ID4_14032025143525.csv
Завантажено новий файл: ./vhi_data\NOAA_ID5_14032025143525.csv
Завантажено новий файл: ./vhi_data\NOAA_ID6_14032025143526.csv
Завантажено новий файл: ./vhi_data\NOAA_ID7_14032025143527.csv
Завантажено новий файл: ./vhi_data\NOAA_ID8_14032025143528.csv
Завантажено новий файл: ./vhi_data\NOAA_ID9_14032025143529.csv
Завантажено новий файл: ./vhi_data\NOAA_ID10_14032025143530.csv
Завантажено новий файл: ./vhi_data\NOAA_ID11_14032025143531.csv
Завантажено новий файл: ./vhi_data\NOAA_ID12_14032025143532.csv
Завантажено новий файл: ./vhi_data\NOAA_ID13_14032025143533.csv
Завантажено новий файл: ./vhi_data\NOAA_ID14_14032025143533.csv
Завантажено новий файл: ./vhi_data\NOAA_ID15_14032025143534.csv
Завантажено новий файл: ./vhi_data\NOAA_ID16_1403

---
### -Створення процедур:

In [3]:
region_name_to_id = {
    "черкаська": 22, "чернігівська": 24, "чернівецька": 23, "крим": 25, "дніпропетровська": 3, "донецька": 4, "івано-франківська": 8, "харківська": 19, 
    "херсонська": 20, "хмельницька": 21, "київська": 9, "київ": 26, "кіровоградська": 10, "луганська": 11, "львівська": 12, "миколаївська": 13, "одеська": 14,
    "полтавська": 15, "рівненська": 16, "севастополь": 27, "сумська": 17, "тернопільська": 18, "закарпатська": 6, "вінницька": 1, "волинська": 2, "запорізька": 7,
    "житомирська": 5
}

region_id_to_name = {v: k.title() for k, v in region_name_to_id.items()}

def get_region_id(region):
    if isinstance(region, int):
        return region
    elif isinstance(region, str):
        region_lower = region.lower()
        if region_lower in region_name_to_id:
            return region_name_to_id[region_lower]
        else:
            raise ValueError(f"Область '{region}' не знайдена у словнику.")
    else:
        raise TypeError("Тип області має бути або int, або str.")

def get_vhi_series_by_year(region, year, df):
    region_id = get_region_id(region)
    region_name = region_id_to_name.get(region_id, f"ID_{region_id}")
    subset = df[(df['area'] == region_id) & (df['Year'] == year) & (df['VHI'] != -1)]
    print(f"\nРяд VHI для області '{region_name}' (айді: {region_id}) за рік {year}")
    if not subset.empty:
        print(subset[['Year', 'Week', 'VHI']])
    else:
        print("Немає даних для заданої області та року.")
    return subset

def get_statistics_for_region_years(regions, years, df):
    if not isinstance(regions, list):
        regions = [regions]
    if not isinstance(years, list):
        years = [years]
        
    region_ids = [get_region_id(r) for r in regions]
    region_names = [region_id_to_name.get(r, f"ID_{r}") for r in region_ids]
    
    subset = df[(df['area'].isin(region_ids)) & (df['Year'].isin(years)) & (df['VHI'] != -1)]
    print(f"\nСтатистика VHI для областей {region_names} (айді: {region_ids}) за роки {years}")
    if not subset.empty:
        vhi_min = subset['VHI'].min()
        vhi_max = subset['VHI'].max()
        vhi_mean = subset['VHI'].mean()
        vhi_median = subset['VHI'].median()
        print("Мінімальне значення VHI:", vhi_min)
        print("Максимальне значення VHI:", vhi_max)
        print("Середнє значення VHI:", vhi_mean)
        print("Медіана VHI:", vhi_median)
        stats = {'min': vhi_min, 'max': vhi_max, 'mean': vhi_mean, 'median': vhi_median}
    else:
        print("Немає даних для заданих областей та років.")
        stats = {}
    return stats

def get_vhi_series_for_regions_year_range(regions, start_year, end_year, df):
    if not isinstance(regions, list):
        regions = [regions]
    region_ids = [get_region_id(r) for r in regions]
    region_names = [region_id_to_name.get(r, f"ID_{r}") for r in region_ids]
    subset = df[(df['area'].isin(region_ids)) & 
                (df['Year'] >= start_year) & (df['Year'] <= end_year) & 
                (df['VHI'] != -1)]
    print(f"\nРяд VHI для областей {region_names} (айді: {region_ids}) за період {start_year}-{end_year}")
    if not subset.empty:
        print(subset[['Year', 'Week', 'area', 'VHI']].sort_values(by=['Year', 'Week']))
    else:
        print("Немає даних для заданих умов.")
    return subset

def find_years_extreme_drought(df, drought_threshold=15, min_percentage=20):
    df_drought = df[(df['VHI'] <= drought_threshold) & (df['VHI'] != -1)]
    
    unique_regions = df['area'].unique()
    total_regions = len(unique_regions)
    required_count = math.ceil((min_percentage / 100) * total_regions)
    
    print(f"\nАналіз екстремальних посух")
    print(f"Визначаються роки, коли посухи торкнулися принаймні {min_percentage}% областей (тобто {required_count} з {total_regions}).")
    
    result = {}
    for year, group in df_drought.groupby('Year'):
        affected_regions = group['area'].unique()
        if len(affected_regions) >= required_count:
            details = group[['area', 'VHI']].copy()
            details['Назва області/міста'] = details['area'].apply(lambda x: region_id_to_name.get(x, f"ID_{x}"))
            result[year] = details.sort_values(by='VHI')
    
    if result:
        print("Знайдені роки з екстремальними посухами:")
        for yr, details in result.items():
            print(f"\nРік: {yr}")
            print(details[['Назва області/міста', 'VHI']])
    else:
        print("Немає років, що відповідають заданим умовам.")
    
    return result
    
print('Необхідні процедури створено без помилок')

Необхідні процедури створено без помилок


---
### -Виконання процедур:

In [20]:
# get_vhi_series_by_year("Київ", 2022, full_df);

In [25]:
# get_statistics_for_region_years(["Київська", "Львівська"], [2010, 2011, 2012], full_df);

In [27]:
# get_vhi_series_for_regions_year_range(["Харківська", "Одеська"], 2005, 2015, full_df);

In [30]:
# find_years_extreme_drought(full_df, drought_threshold=15, min_percentage=20);